In [7]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [8]:

import sys

import ipywidgets as widgets

sys.path.append('../')
sys.path.append('../src')

import src.backend
import src.workload_builder as builder
from mbi import Domain
import src.plots
import altair as alt
alt.renderers.enable('default')
from IPython.display import display, clear_output

alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

In [9]:
data_path = '../data/CPS/CPS.csv'
cps_domain = Domain(attrs=('age','income','marital','race'), shape=(100,100,7,4))
questions = ["Warm Up: Type 'Yes' if you are ready to begin!",
            "Tutorial Question 1/6: How many Chinese and American Indians have 'Widowed' as their marital status?",
            "Tutorial Question 2/6: How many people over the age of 65 have income between 100,000 and 149,000?",
            "Tutorial Question 3/6: What is the second most common marital status of people with income in the 100,000 to 199,000 range?",
            "Tutorial Question 4/6: Choose the top 3 age groups that have marital status: 'Never Married'?",
            "Tutorial Question 5/6: In the 100,000 to 199,000 income range, rank the top 3 races in terms of count from greatest to least.",
            "Tutorial Question 6/6: What is the second most common race in the 45-54 age group?"]

marital_race = {'marital':1, 'race':1}
age_income = {'age':10, 'income':10}
marital_income = {'income':10, 'marital':1}
age_marital = {'age':10, 'marital':1}
race_income = {'income':10, 'race':1}
age_race = {'age':10, 'race':1}

visualizations = [None, marital_race, age_income, marital_income, age_marital, race_income, age_race]
prev_spec = None
curr_spec = None
epsilon_increments = [None,0.05,0.05,0.05,0.05,0.05,0.05,0.05]
budget = 1.0
max_tries = 10

index = 0
tries = 0
back_end = None


In [10]:
def binning(measure_dict, id=None, epsilon=None):
    global tries, curr_spec, back_end, current_spec_arr, hist_workloads

    left_col = measure_dict['left']
    right_col = measure_dict['right']

    widths = {left_col[0]:left_col[1], right_col[0]:right_col[1]}

    hist = builder.histogram_workload(cps_domain.config, bin_widths=widths)

    y_hat, strategy_matrix = back_end.measure_hdmm(workload=hist, eps=epsilon, restarts=20)

    prev_spec = current_spec_arr[id]
    # print(prev_spec)
    prev_spec.rename(columns={'error': 'error_prev', 'plus_error': 'plus_error_prev', 'minus_error': 'minus_error_prev', 'true_count':'true_count_prev', 'noisy_count':'noisy_count_prev'}, inplace=True)

    current_spec_arr[id], cached = back_end.display(hist) ################# Cache_search function called

    spec = current_spec_arr[id].join(prev_spec, on=[left_col[0], right_col[0]]).reset_index([left_col[0], right_col[0]])
    return spec
    # spec = spec.round(0)  We should not round

In [11]:
num_answers = len(questions)

current_spec_arr = []
hist_workloads = []

def make_vis(do_not_update=None):
    global index, visualizations, epsilon_increments, curr_spec, back_end
    back_end = src.backend.initialize_backend(cps_domain, data_path, budget=1.0)

    for i in range(len(questions) - 1):

        hist = builder.histogram_workload(cps_domain.config, bin_widths=visualizations[i+1])
        hist_workloads.append(hist)

        y_hat, strategy_matrix = back_end.measure_hdmm(workload=hist, eps=epsilon_increments[i+1], restarts=20)

        column_names = list(visualizations[i+1].keys())
        curr_spec, cached = back_end.display(hist)
        current_spec_arr.append(curr_spec)

    return current_spec_arr

curr_specs_for_visualizations = make_vis()

def apply_multiple_remeasures(no_of_remeasures, query_no):
    ans_values = []
    key_val = [x for x in visualizations[query_no].items()]
    measure_dict = {'left': key_val[0], 'right': key_val[1]}
    for i in range(no_of_remeasures):
        spec_after_remeasure = binning(measure_dict, query_no - 1, epsilon_increments[query_no])
        if query_no == 1:
            ans = spec_after_remeasure.loc[(spec_after_remeasure['marital'] == 'Widowed') & ((spec_after_remeasure['race'] == 'American Indian')|(spec_after_remeasure['race'] == 'Chinese'))]
            ans_values.append(ans['noisy_count'].sum())
        elif query_no == 2:
            selected_rows = spec_after_remeasure[spec_after_remeasure['age'].apply(lambda x: int(x.split('-')[0])) >= 65]
            ans = selected_rows.loc[(selected_rows['income'] == '$100k - $149k')]
            ans_values.append(ans['noisy_count'].sum())
        elif query_no == 3:
            ans1 = spec_after_remeasure.loc[(spec_after_remeasure['income'] == '$100k - $149k')]
            ans2 = spec_after_remeasure.loc[(spec_after_remeasure['income'] == '$150k - $199k')]
            ans = ans1.append(ans2)
            sum_dict = {}
            s_married = ans.loc[(ans['marital'] == 'Married')]
            sum_dict['Married'] = s_married['noisy_count'].sum()
            s_absent = ans.loc[(ans['marital'] == 'Absent')]
            sum_dict['Absent'] = s_absent['noisy_count'].sum()
            s_separated = ans.loc[(ans['marital'] == 'Separated')]
            sum_dict['Separated'] = s_separated['noisy_count'].sum()
            s_widowed = ans.loc[(ans['marital'] == 'Widowed')]
            sum_dict['Widowed'] = s_widowed['noisy_count'].sum()
            s_divorced = ans.loc[(ans['marital'] == 'Divorced')]
            sum_dict['Divorced'] = s_divorced['noisy_count'].sum()
            s_div_wid = ans.loc[(ans['marital'] == 'Divorced/Widowed')]
            sum_dict['Divorced/Widowed'] = s_div_wid['noisy_count'].sum()
            result = sorted(sum_dict.items(), key=lambda x:x[1])
            ans_values.append(result[-2])
        elif query_no == 4:
            ans = spec_after_remeasure.loc[(spec_after_remeasure['marital'] == 'Never Married')]
            sorted_ans = ans.sort_values('noisy_count')
            top_three_age_groups = [sorted_ans['age'].iloc[-1], sorted_ans['age'].iloc[-2], sorted_ans['age'].iloc[-3]]
            ans_values.append(top_three_age_groups)
        elif query_no == 5:
            ans1 = spec_after_remeasure.loc[(spec_after_remeasure['income'] == '$100k - $149k')]
            ans2 = spec_after_remeasure.loc[(spec_after_remeasure['income'] == '$150k - $199k')]
            ans = ans1.append(ans2)
            sum_dict = {}
            s_white = ans.loc[(ans['race'] == 'White')]
            sum_dict['White'] = s_white['noisy_count'].sum()
            s_black = ans.loc[(ans['race'] == 'Black')]
            sum_dict['Black'] = s_black['noisy_count'].sum()
            s_american_indian = ans.loc[(ans['race'] == 'American Indian')]
            sum_dict['American Indian'] = s_american_indian['noisy_count'].sum()
            s_chinese = ans.loc[(ans['race'] == 'Chinese')]
            sum_dict['Chinese'] = s_chinese['noisy_count'].sum()
            result = sorted(sum_dict.items(), key=lambda x:x[1])
            top_three_races = [result[-1], result[-2], result[-3]]
            ans_values.append(top_three_races)
        elif query_no == 6:
            ans = spec_after_remeasure.loc[(spec_after_remeasure['age'] == '45-54')]
            sorted_ans = ans.sort_values('noisy_count')
            second_most_common_race = sorted_ans['race'].iloc[-2]
            ans_values.append(second_most_common_race)
    return ans_values[no_of_remeasures - 1]

number_of_remeasure = 3
query_number = 1
output = apply_multiple_remeasures(number_of_remeasure, query_number)


[('age', 10), ('income', 10)]
        age         income  noisy_count  true_count  plus_error  minus_error  \
0     15-24     $0k - $49k       1411.1      1474.0      1486.0       1336.1   
1     15-24    $50k - $99k        162.5        73.0       304.0         21.1   
2     15-24  $100k - $149k         62.9         6.0       131.1         -5.3   
3     15-24  $150k - $199k       -353.0         1.0      -211.6       -494.4   
4     15-24  $200k - $249k         19.2         2.0        87.6        -49.2   
..      ...            ...          ...         ...         ...          ...   
95  105-114  $250k - $299k         -5.3         0.0        65.9        -76.4   
96  105-114  $300k - $349k       -142.6         0.0       -65.1       -220.2   
97  105-114  $350k - $399k        -89.7         0.0       -21.5       -157.9   
98  105-114  $400k - $449k       -130.0         0.0        11.4       -271.4   
99  105-114  $450k - $499k        162.1         0.0       231.1         93.1   

    error

In [12]:
# key_val = [x for x in visualizations[1].items()]
# print("keyyyyy   ", key_val)
# measure_dict = {'left': key_val[0], 'right': key_val[1]}
# binning(measure_dict, 0, epsilon = epsilon_increments[1])